In [1]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
torch.__version__  

'1.9.1+cpu'

In [4]:
import os
root = 'data_ABIDE_useful (2)'
label_list = os.listdir(root)
label_list

['ASD', 'HC']

In [5]:

class_dict = {
    "HC": 0,
    "ASD": 1,
}
threshold = 0.2
data_processed = []
for label_files in label_list:
    label = torch.LongTensor([class_dict[label_files]])
    listd = os.listdir(os.path.join(root, label_files))
    for files in listd:
        subj_fc_dir = os.path.join(root, label_files, files)
        subj_mat_fc = np.loadtxt(subj_fc_dir)[:176, :90]
        #print("reading data " + subj_fc_dir)
        subj_mat_fc_adj = np.corrcoef(np.transpose(subj_mat_fc))  #90*90--correlation coefficient
        subj_mat_fc_adj = subj_mat_fc_adj - np.diag(np.diag(subj_mat_fc_adj))
        #take the upper triangle and compute the threshold
        subj_fc_adj_up = subj_mat_fc_adj[np.triu_indices(90, k=1)]
        subj_fc_adj_list = subj_fc_adj_up.reshape((-1))
        thindex = int(threshold * subj_fc_adj_list.shape[0])
        thremax = subj_fc_adj_list[subj_fc_adj_list.argsort()[-1 * thindex - 1]]  #
        #avoiding Nan
        subj_fc_adj_t = np.zeros((90, 90))
        subj_fc_adj_t[subj_mat_fc_adj > thremax] = 1
        subj_mat_fc_adj = subj_fc_adj_t
        fcedge_index, _ = dense_to_sparse(torch.from_numpy(subj_mat_fc_adj.astype(np.int16)))

        subj_mat_fc_list = subj_mat_fc.reshape((-1))
        subj_mat_fc_new = (subj_mat_fc - min(subj_mat_fc_list)) / (
                max(subj_mat_fc_list) - min(subj_mat_fc_list))
        # subj_mat_fc_new = (subj_mat_fc - np.mean(subj_mat_fc, axis=0, keepdims=True)) / np.std(subj_mat_fc, axis=0, keepdims=True)

        subj_mat_fc_new = np.transpose(subj_mat_fc_new)  #90*176

        rowsum = np.array(subj_mat_fc_adj.sum(1))  #row sum #90,
        N = np.diag(rowsum)  #90*90
        degree_C_BOLD = np.concatenate((N, subj_mat_fc_new), 1)  #90*266
        # print(subj_mat_fc_adj)
        # BOLD_C_degree=np.concatenate((subj_mat_fc_new,N),1)
        ###one-hot###
        # subj_mat_fc_new=np.eye(args.num_nodes)
        data_processed.append(
            Data(x=torch.from_numpy(degree_C_BOLD).float(), edge_index=fcedge_index, y=torch.tensor(label)))


[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ...

D:\Software\anaconda\envs\autism\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[[0. 0. 1. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ...

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 1. 0. ... 1. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 1. 0.]]
[[0. 1. 1. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ...

[[0. 1. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ...

...

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 0. 0.]]
[[0. 1. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 1.]
 [1. 1. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]
 [0. 1. 0. ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 0. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 1. 1. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 1. 0. 1.]
 [1. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 1. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

In [6]:
import random
random.seed(1234)
random.shuffle(data_processed)
data_processed #len:618; 

[Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1603], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1603], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1603], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1603], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2, 1602], x=[90, 266], y=[1]),
 Data(edge_index=[2,

In [95]:
data_processed[0].x.shape[1]

266

500

In [184]:
train_dataset = data_processed[:500]
test_dataset = data_processed[500:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 500
Number of test graphs: 118


In [15]:
import torch_geometric
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

#self.register_buffer('GCN', torch.full((len(vocab), ), fill_value=unknown_idx, dtype=torch.long))

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102538], ptr=[65], x=[5760, 266], y=[64])

Step 2:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102544], ptr=[65], x=[5760, 266], y=[64])

Step 3:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102541], ptr=[65], x=[5760, 266], y=[64])

Step 4:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102536], ptr=[65], x=[5760, 266], y=[64])

Step 5:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102544], ptr=[65], x=[5760, 266], y=[64])

Step 6:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102543], ptr=[65], x=[5760, 266], y=[64])

Step 7:
Number of graphs in the current batch: 64
Batch(batch=[5760], edge_index=[2, 102543], ptr=[65], x=[5760, 266], y=[64])

Step 8:
Number of graphs in the current batch: 52
Batch(batch=[4680], edge_index=[2, 83315], ptr=[53], x

## training directly by covariance

In [2]:
import os
root = 'data_ABIDE_useful (2)'
label_list = os.listdir(root)
import numpy as np
import pandas as pd
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import Data

class_dict = {
            "HC": 0,
            "ASD": 1,
            }
threshold = 0.2
data_processed = []
for label_files in label_list:
    label = torch.LongTensor([class_dict[label_files]])
    list = os.listdir(os.path.join(root, label_files))
    for files in list:
        subj_fc_dir = os.path.join(root, label_files, files)
        subj_mat_fc=np.loadtxt(subj_fc_dir)[:176,:90]
        #print("reading data " + subj_fc_dir)
        subj_mat_fc_adj = np.corrcoef(np.transpose(subj_mat_fc))#90*90--correlation coefficient
        subj_mat_fc_adj = subj_mat_fc_adj - np.diag(np.diag(subj_mat_fc_adj))
        #print(subj_mat_fc_adj.shape)
        corr=subj_mat_fc_adj
        #take the upper triangle and compute the threshold
        subj_fc_adj_up=subj_mat_fc_adj[np.triu_indices(90,k=1)]
        subj_fc_adj_list = subj_fc_adj_up.reshape((-1))
        thindex = int(threshold * subj_fc_adj_list.shape[0])
        thremax = subj_fc_adj_list[subj_fc_adj_list.argsort()[-1 * thindex-1]]#
        #avoiding Nan
        subj_fc_adj_t = np.zeros((90, 90))
        subj_fc_adj_t[subj_mat_fc_adj > thremax] = 1
        subj_mat_fc_adj=subj_fc_adj_t
        fcedge_index, _ = dense_to_sparse(torch.from_numpy(subj_mat_fc_adj.astype(np.int16)))

        subj_mat_fc_list = subj_mat_fc.reshape((-1))
        subj_mat_fc_new = (subj_mat_fc - min(subj_mat_fc_list)) / (
                max(subj_mat_fc_list) - min(subj_mat_fc_list))
        # subj_mat_fc_new = (subj_mat_fc - np.mean(subj_mat_fc, axis=0, keepdims=True)) / np.std(subj_mat_fc, axis=0, keepdims=True)

        subj_mat_fc_new = np.transpose(subj_mat_fc_new)#90*176
                
        rowsum = np.array(subj_mat_fc_adj.sum(1))#row sum #90,
        N = np.diag(rowsum)#90*90
        degree_C_BOLD=np.concatenate((N,subj_mat_fc_new),1)#90*266
        #print(type(fcedge_index))
        #print(subj_mat_fc_adj)
        BOLD_C_degree=np.concatenate((subj_mat_fc_new,N),1)
        #print(type(BOLD_C_degree))
        ###one-hot###
        # subj_mat_fc_new=np.eye(args.num_nodes)
        data_processed.append(Data(x=torch.from_numpy(corr).float(),edge_index=fcedge_index, y=torch.tensor(label)))
        
import random
random.seed(1234)
random.shuffle(data_processed)
train_dataset = data_processed[:500]
test_dataset = data_processed[500:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

D:\Software\anaconda\envs\autism\lib\site-packages\ipykernel_launcher.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Number of training graphs: 500
Number of test graphs: 118


In [3]:
import torch_geometric
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51273], ptr=[33], x=[2880, 90], y=[32])

Step 2:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51268], ptr=[33], x=[2880, 90], y=[32])

Step 3:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51272], ptr=[33], x=[2880, 90], y=[32])

Step 4:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51269], ptr=[33], x=[2880, 90], y=[32])

Step 5:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51272], ptr=[33], x=[2880, 90], y=[32])

Step 6:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51276], ptr=[33], x=[2880, 90], y=[32])

Step 7:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51267], ptr=[33], x=[2880, 90], y=[32])

Step 8:
Number of graphs in the current batch: 32
Batch(batch=[2880], edge_index=[2, 51274], ptr=[33], x=[2880, 90], y

In [5]:
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(data_processed[0].x.shape[1], hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GNN(hidden_channels=64)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            # batch iteration of test set data set
        out = model(data.x, data.edge_index, data.batch) # one feed forward
        pred = out.argmax(dim=1)                         # use category with highest probability
        correct += int((pred == data.y).sum())           # Check ground-truth labels
    return correct / len(loader.dataset)


for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

GNN(
  (conv1): GraphConv(90, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
Epoch: 001, Train Acc: 0.5160, Test Acc: 0.4831
Epoch: 002, Train Acc: 0.5180, Test Acc: 0.5000
Epoch: 003, Train Acc: 0.5460, Test Acc: 0.5085
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.5000
Epoch: 005, Train Acc: 0.5780, Test Acc: 0.5085
Epoch: 006, Train Acc: 0.5900, Test Acc: 0.5424
Epoch: 007, Train Acc: 0.5900, Test Acc: 0.4915
Epoch: 008, Train Acc: 0.5940, Test Acc: 0.4831
Epoch: 009, Train Acc: 0.5900, Test Acc: 0.5593
Epoch: 010, Train Acc: 0.5980, Test Acc: 0.5678
Epoch: 011, Train Acc: 0.6160, Test Acc: 0.5593
Epoch: 012, Train Acc: 0.6440, Test Acc: 0.5763
Epoch: 013, Train Acc: 0.6360, Test Acc: 0.5593
Epoch: 014, Train Acc: 0.6340, Test Acc: 0.5763
Epoch: 015, Train Acc: 0.6440, Test Acc: 0.5508
Epoch: 016, Train Acc: 0.6560, Test Acc: 0.5593
Epoch: 017, Train Acc: 0.6640, Test Acc: 0.5763
Epoch: 018, Train Acc: 0.6660, 

Epoch: 169, Train Acc: 0.9980, Test Acc: 0.5593
Epoch: 170, Train Acc: 0.9980, Test Acc: 0.5593
Epoch: 171, Train Acc: 0.9980, Test Acc: 0.5508
Epoch: 172, Train Acc: 0.9980, Test Acc: 0.5508
Epoch: 173, Train Acc: 0.9980, Test Acc: 0.5508
Epoch: 174, Train Acc: 0.9980, Test Acc: 0.5593
Epoch: 175, Train Acc: 0.9980, Test Acc: 0.5508
Epoch: 176, Train Acc: 0.9980, Test Acc: 0.5593
Epoch: 177, Train Acc: 0.9980, Test Acc: 0.5593
Epoch: 178, Train Acc: 0.9980, Test Acc: 0.5508
Epoch: 179, Train Acc: 0.9980, Test Acc: 0.5508
Epoch: 180, Train Acc: 1.0000, Test Acc: 0.5678
Epoch: 181, Train Acc: 1.0000, Test Acc: 0.5763
Epoch: 182, Train Acc: 1.0000, Test Acc: 0.5508
Epoch: 183, Train Acc: 1.0000, Test Acc: 0.5339
Epoch: 184, Train Acc: 1.0000, Test Acc: 0.5508
Epoch: 185, Train Acc: 1.0000, Test Acc: 0.5424
Epoch: 186, Train Acc: 1.0000, Test Acc: 0.5339
Epoch: 187, Train Acc: 1.0000, Test Acc: 0.5593
Epoch: 188, Train Acc: 1.0000, Test Acc: 0.5763
Epoch: 189, Train Acc: 1.0000, Test Acc:

## train by GCN model

In [4]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(data_processed[0].x.shape[1], hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)
        
    def forward(self, x, edge_index, batch):
        # 1. get node embedding
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)   # [batch_size, hidden_channels]
        
        # 3. classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(90, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [18]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            
        out = model(data.x, data.edge_index, data.batch) 
        pred = out.argmax(dim=1)                         
        correct += int((pred == data.y).sum())           
    return correct / len(loader.dataset)

for epoch in range(1, 121):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 002, Train Acc: 0.4560, Test Acc: 0.5254
Epoch: 003, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 004, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 005, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 006, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 007, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 008, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 009, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 010, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 011, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 012, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 013, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 014, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 015, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 016, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 017, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 018, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 019, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 020, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 021, Train Acc: 0.5440, Test Acc:

In [111]:
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(data_processed[0].x.shape[1], hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GNN(hidden_channels=64)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            
        out = model(data.x, data.edge_index, data.batch) 
        pred = out.argmax(dim=1)                         
        correct += int((pred == data.y).sum())          
    return correct / len(loader.dataset)


for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')


GNN(
  (conv1): GraphConv(266, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
Epoch: 001, Train Acc: 0.4760, Test Acc: 0.5169
Epoch: 002, Train Acc: 0.5680, Test Acc: 0.4746
Epoch: 003, Train Acc: 0.5660, Test Acc: 0.4915
Epoch: 004, Train Acc: 0.5080, Test Acc: 0.4576
Epoch: 005, Train Acc: 0.5760, Test Acc: 0.4407
Epoch: 006, Train Acc: 0.5640, Test Acc: 0.4576
Epoch: 007, Train Acc: 0.5700, Test Acc: 0.4576
Epoch: 008, Train Acc: 0.5860, Test Acc: 0.4746
Epoch: 009, Train Acc: 0.6160, Test Acc: 0.4831
Epoch: 010, Train Acc: 0.6320, Test Acc: 0.5254
Epoch: 011, Train Acc: 0.6240, Test Acc: 0.5424
Epoch: 012, Train Acc: 0.5800, Test Acc: 0.5254
Epoch: 013, Train Acc: 0.5940, Test Acc: 0.5169
Epoch: 014, Train Acc: 0.5800, Test Acc: 0.5339
Epoch: 015, Train Acc: 0.6300, Test Acc: 0.5424
Epoch: 016, Train Acc: 0.6180, Test Acc: 0.5508
Epoch: 017, Train Acc: 0.6340, Test Acc: 0.5254
Epoch: 018, Train Acc: 0.6320,

Epoch: 169, Train Acc: 0.9040, Test Acc: 0.5763
Epoch: 170, Train Acc: 0.9160, Test Acc: 0.5678
Epoch: 171, Train Acc: 0.9060, Test Acc: 0.6102
Epoch: 172, Train Acc: 0.9200, Test Acc: 0.5763
Epoch: 173, Train Acc: 0.8960, Test Acc: 0.5932
Epoch: 174, Train Acc: 0.9160, Test Acc: 0.5763
Epoch: 175, Train Acc: 0.9240, Test Acc: 0.5847
Epoch: 176, Train Acc: 0.9240, Test Acc: 0.5932
Epoch: 177, Train Acc: 0.9260, Test Acc: 0.5763
Epoch: 178, Train Acc: 0.9260, Test Acc: 0.6017
Epoch: 179, Train Acc: 0.9380, Test Acc: 0.5763
Epoch: 180, Train Acc: 0.9280, Test Acc: 0.5678
Epoch: 181, Train Acc: 0.9340, Test Acc: 0.5763
Epoch: 182, Train Acc: 0.9420, Test Acc: 0.5932
Epoch: 183, Train Acc: 0.9300, Test Acc: 0.5678
Epoch: 184, Train Acc: 0.9280, Test Acc: 0.5508
Epoch: 185, Train Acc: 0.9420, Test Acc: 0.5932
Epoch: 186, Train Acc: 0.9420, Test Acc: 0.5847
Epoch: 187, Train Acc: 0.9380, Test Acc: 0.5678
Epoch: 188, Train Acc: 0.9500, Test Acc: 0.5763
Epoch: 189, Train Acc: 0.9440, Test Acc:

In [114]:
model_t=model

In [187]:
a=train_dataset.copy()
len(a)

500

In [197]:
'''
def get_kfold_data(k, i, X):  
     
    fold_size = int(len(X) / k) 
    
    val_start = int((i-1) * fold_size)
    
    val_end = int(i * fold_size)
    if i==1:
        X_valid=X[0:fold_size]
        X_train=X.copy()
        del X_train[0:fold_size]
    elif i>1 and i<k:   
        X_valid=X[val_start:val_end]
        X_train=X.copy()
        del X_train[val_start:val_end]
        print("len:",len(X_train),len(X_valid))
    elif i==k:
        X_valid=X[val_start:-1]
        X_train=X.copy()
        del X_train[val_start:]
    return X_train, X_valid

from torch_geometric.nn import GraphConv

'''
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(train_dataset[0].x.shape[1], hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x
'''    
    
def train():
    model.train()
    
    for data in loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader,model):
    model.eval()
    
    correct = 0
    for data in loader:                            
        out = model(data.x, data.edge_index, data.batch) 
        pred = out.argmax(dim=1)                         
        correct += int((pred == data.y).sum())           
    return correct / len(loader)

    
def traink(model, X_train, X_val,EPOCH):
    train_lo = DataLoader(X_train, batch_size=64, shuffle=True)
    test_lo = DataLoader(X_val, batch_size=64, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    criterion = nn.CrossEntropyLoss()
    
    print("lo:",len(train_lo),len(test_lo))
    
    train_acc = []
    val_acc = []
     
    for epoch in range(EPOCH):
        model.train()

        for data in train_lo:
            optimizer.zero_grad()

            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)

            loss.backward()
            optimizer.step()
            
        correct = 0   # record the correct number and print accuracy after each epoch training is completed
        train_acc = test(train_lo,model)
        test_acc = test(test_lo,model)

        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

    return train_acc, test_acc#losses, val_losses, 

def k_fold(k, train_dataset,EPOCH,model):
    
    train_acc_sum , valid_acc_sum = 0, 0
    
    for i in range(k):
        i=i+1
        print('*'*25,'第', i,'折','*'*25)
        data = get_kfold_data(k, i, train_dataset)    # get training and validation data for k-fold cross validation
        #print(data)
        net = model  # instantiate the model (an already defined model)
        # train on each piece of data
        train_acc, val_acc = traink(net, *data,EPOCH) #train_loss, val_loss,
        print(val_acc,type(train_acc))
        
        #print('train_loss:{:.5f}, train_acc:{:.3f}%'.format(train_loss[-1], train_acc[-1]))
        #print('valid loss:{:.5f}, valid_acc:{:.3f}%\n'.format(val_loss[-1], val_acc[-1]))
        #print(' train_acc:{:.3f}%'.format( train_acc[-1]))
        #print(' valid_acc:{:.3f}%\n'.format( val_acc[-1]))
        
        #train_loss_sum += train_loss[-1]
        #valid_loss_sum += val_loss[-1]
        train_acc_sum += train_acc
        valid_acc_sum += val_acc
        
    print('\n', '#'*10,'最终k折交叉验证结果','#'*10) # print the final result for k-fold
    

    #print('average train loss:{:.4f}, average train accuracy:{:.3f}%'.format(train_loss_sum/k, train_acc_sum/k))
    #print('average valid loss:{:.4f}, average valid accuracy:{:.3f}%'.format(valid_loss_sum/k, valid_acc_sum/k))
    print(' average train accuracy:{:.3f}%'.format( train_acc_sum/k))
    print('average valid accuracy:{:.3f}%'.format( valid_acc_sum/k))

    return 

k_fold(10,train_dataset,10,model_t)

************************* 第 1 折 *************************
lo: 8 1
Epoch: 000, Train Acc: 47.6250, Test Acc: 35.0000
Epoch: 001, Train Acc: 47.7500, Test Acc: 38.0000
Epoch: 002, Train Acc: 48.6250, Test Acc: 41.0000
Epoch: 003, Train Acc: 50.0000, Test Acc: 42.0000
Epoch: 004, Train Acc: 50.8750, Test Acc: 43.0000
Epoch: 005, Train Acc: 51.3750, Test Acc: 44.0000
Epoch: 006, Train Acc: 51.1250, Test Acc: 42.0000
Epoch: 007, Train Acc: 51.1250, Test Acc: 45.0000
Epoch: 008, Train Acc: 52.2500, Test Acc: 42.0000
Epoch: 009, Train Acc: 52.0000, Test Acc: 45.0000
45.0 <class 'float'>
************************* 第 2 折 *************************
len: 450 50
lo: 8 1
Epoch: 000, Train Acc: 52.8750, Test Acc: 45.0000
Epoch: 001, Train Acc: 53.3750, Test Acc: 46.0000
Epoch: 002, Train Acc: 53.0000, Test Acc: 44.0000
Epoch: 003, Train Acc: 53.1250, Test Acc: 45.0000
Epoch: 004, Train Acc: 53.6250, Test Acc: 45.0000
Epoch: 005, Train Acc: 54.1250, Test Acc: 46.0000
Epoch: 006, Train Acc: 53.8750, Tes

KeyboardInterrupt: 